In [ ]:
# Refer to the file readme.txt for isntructions to run this file. Only the block below is to be changed.

In [ ]:
# enter the training file name.
file = ''
# enter the number of features.
numfeatures = 
# enter the test set size.
test_set = 

In [ ]:
# form the dataset by getting corresponding features and execution time.
dataset = []
import numpy as np
with open(file) as f:
    lines = f.readlines()
    for l in lines:
        l = l[:-1]
        l = l.split(' ')
        dataset.append([float(numeric_string) for numeric_string in l])
dataset = np.asarray(dataset)
print(len(dataset))

In [ ]:
# function to normalize the data
def normalize(train):
    train_norm = np.asarray([[0.0]*len(train[0])]*len(train))
    minTrain = train.min(axis = 0)
    maxTrain = train.max(axis = 0)
    ranges = maxTrain - minTrain
    for i in range(len(train[0])):
        for j in range(len(train)):
            if(ranges[i] == 0):
                train_norm[j][i] = 0.0
            else:
                train_norm[j][i] = (train[j][i] - minTrain[i])/ranges[i]
    return train_norm
temp = normalize(np.asarray(dataset[:, :-1])) 
dataset = np.concatenate((temp, dataset[:,-1].reshape(-1,1)), axis=1)

In [ ]:
# dividing dataset into training and test set
dataset.sort()
print(dataset.shape)
train_data = dataset[:-1*test_set]
test_data = dataset[-1*test_set:]

In [ ]:
np.random.seed(7)
train_data = np.asarray(train_data)

In [ ]:
# separating labels from test and train set
train = train_data[:, :-1]
label_train = train_data[:,-1:]
test = test_data[:, :-1]
label_test = test_data[:,-1:]

In [ ]:
# importing and running the svm model
from sklearn import svm
from sklearn import preprocessing

In [ ]:
model = svm.SVR(kernel='rbf', gamma=1, max_iter=1000)
label_train = label_train.reshape(len(label_train))
model.fit(train,label_train)

In [ ]:
result = model.predict(test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(result,label_test.reshape(label_test.shape[0]))

In [ ]:
# importing and running the linear regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(train,label_train)

In [ ]:
result = model.predict(test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(result,label_test.reshape(label_test.shape[0]))

In [ ]:
# implementing and running the neural network model
import torch
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(numfeatures, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 1),
            nn.LeakyReLU()
        )
        self.double()

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork()
print(model)

In [ ]:
from torch.optim import Adam
criterion = nn.MSELoss()
EPOCHS = 1000
optm = Adam(model.parameters(), lr = 0.001)

In [ ]:
def training(model, x, y, optimizer, criterion):
    model.zero_grad()
    output = model(x)
    output = output.reshape(-1)
    y= torch.tensor(y)
    y = y.reshape(1)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()

    return loss, output

In [ ]:
for epoch in range(EPOCHS):
    epoch_loss = 0
    model.train()
    for i in range(len(train)):
        x_train, y_train = train[i], label_train[i]
        x_train = torch.from_numpy(x_train)
        loss, predictions = training(model,x_train,y_train, optm, criterion)
        epoch_loss+=loss.item()
    print('Epoch {} Loss : {}'.format((epoch+1),epoch_loss))

In [ ]:
model.eval()
result = []
for i in range(len(test)):
    result.append(model(torch.from_numpy(test[i])).detach().numpy())
mean_squared_error(result,label_test.reshape(label_test.shape[0]))